In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import seaborn as sns

In [8]:
#Pull in base dataframe and add avg departure delay
df = pd.read_csv('datasets/test_flights_40_percent.csv')
dep_delay = pd.read_csv('../CleaningTableFlights/data/flights.csv')[['dep_delay','origin']]
arr_delay = pd.read_csv('../CleaningTableFlights/data/flights.csv')[['arr_delay','dest']]

In [9]:
#departure delay
dep_delay = dep_delay.groupby('origin').mean()
dep_delay = dep_delay.rename({'dep_delay':'average_departure_delay_by_ap'},axis=1)

In [10]:
df

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,15865443,2018-07-11,B6,B6,B6,583,B6,N950JB,583,12478,...,944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4508273,2019-10-02,B6,B6,B6,382,B6,N239JB,382,14524,...,474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3645683,2018-01-25,AA,AA_CODESHARE,AA,4529,YX,N124HQ,4529,11278,...,204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13782805,2018-02-01,DL,DL,DL,1462,DL,N906DE,1462,14122,...,526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8880630,2018-06-19,DL,DL,DL,1180,DL,N925DN,1180,11193,...,1069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252954,3143816,2019-07-30,AA,AA,AA,2274,AA,N9002U,2274,11298,...,1212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252955,4187672,2018-08-19,WN,WN,WN,6999,WN,N7709A,6999,10397,...,591,11.0,0.0,0.0,0.0,11.0,NaN,NaN,NaN,NaN
252956,14020088,2018-02-13,AS,AS,AS,510,AS,N508AS,510,14747,...,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252957,5328525,2018-11-08,DL,DL,DL,2827,DL,N394DA,2827,10397,...,373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#add constant
df['constant'] = 1
first_column = df.pop('constant')
df.insert(0, 'constant', first_column)

In [12]:
#arr_delay data by airport
arr_delay = arr_delay.groupby('dest').mean()
arr_delay = arr_delay.rename({'arr_delay':'average_arrival_delay_by_ap_full'},axis=1)

In [13]:
arr_delay.head()

,average_arrival_delay_by_ap_full
dest,
ABE,6.937888
ABI,-1.176471
ABQ,6.361068
ABR,0.818182
ABY,6.000000


In [20]:
#passenger_fuel_info by carrier
pass_fuel_df = pd.read_csv('data/new_features/passenger_fuel_df.csv')
pass_fuel_df.head()

,mkt_unique_carrier,avg_distance_per_month_by_carrier,avg_dep_delay_by_carrier,total_gallons,passengers_by_carrier_per_month,monthly_distance_per_passenger,avgfuel_percustomer_perdistance
0,AA,9.001721e+07,1.280473e+06,529073.835069,1.269777e+07,0.141059,3.750717e+06
1,AS,2.656357e+07,9.769804e+04,121682.671875,2.920384e+06,0.109939,1.106816e+06
2,B6,2.069899e+07,3.156945e+05,147726.593750,3.545438e+06,0.171286,8.624578e+05
3,DL,7.740976e+07,9.859202e+05,547151.206597,1.313163e+07,0.169638,3.225407e+06
4,F9,8.300151e+06,1.352600e+05,74017.159722,1.776412e+06,0.214022,3.458396e+05


In [21]:
#departures info by year per airport
departures_df = pd.read_csv('data/departures_by_airport_full.csv')
departures_df = departures_df.drop('origin_city_name',axis=1)
departures_df.head()

,origin,yearly_departures_per_ap,yearly_passengers_per_ap
0,05A,75.4,149.6
1,06A,7.2,3.8
2,08A,2.0,3.0
3,09A,7.2,8.6
4,1B1,1.8,5.8


In [22]:
#arrival info by year per airport
arrivals_df = pd.read_csv('data/arrivals_by_airport.csv')
arrivals_df = arrivals_df.drop('dest_city_name',axis=1)
arrivals_df.head()

,dest,yearly_arrivals_per_ap,yearly_arriving_passengers_per_ap
0,05A,76.6,168.6
1,06A,7.0,3.6
2,08A,2.2,3.4
3,09A,7.4,13.0
4,1B1,1.2,3.2


In [23]:
#monthly destination and origin info
month_dest_df = pd.read_csv('data/new_features/features_dest_monthly_params.csv')
month_origin_df = pd.read_csv('data/new_features/features_origin_monthly_params.csv')

In [24]:
#create month and weekday variables
df['month'] = pd.DatetimeIndex(df['fl_date']).month
df['weekday'] = pd.DatetimeIndex(df['fl_date']).weekday

In [25]:
#create combo variables for grouping
month_dest_df['combo1']=month_dest_df['dest'].astype('str')+month_dest_df['fl_month'].astype('str')
df['combo1']=df['dest'].astype('str')+df['month'].astype('str')

#rename columns for clarity and drop useless columns
month_dest_df = month_dest_df.drop(['fl_month','dest'],axis=1)
month_dest_df = month_dest_df.rename(columns={'nas_delay':'avg_nas_delay_by_month_by_dest_ap','carrier_delay':'avg_carrier_delay_by_month_by_dest_ap','arr_delay':'avg_arr_delay_by_month_by_dest_ap','cancelled':'cancelled_flights_by_month_by_dest_ap','weather_delay':'avg_weather_delay_by_month_by_dest_ap','security_delay':'security_delay_by_month_by_dest_ap'})
month_dest_df.head()

,avg_arr_delay_by_month_by_dest_ap,avg_carrier_delay_by_month_by_dest_ap,avg_nas_delay_by_month_by_dest_ap,late_aircraft_delay,avg_weather_delay_by_month_by_dest_ap,security_delay_by_month_by_dest_ap,cancelled_flights_by_month_by_dest_ap,combo1


In [26]:
#create combo variables for grouping
month_origin_df['combo2']=month_origin_df['origin'].astype('str')+month_origin_df['fl_month'].astype('str')
df['combo2']=df['origin'].astype('str')+df['month'].astype('str')

#rename columns for clarity and drop useless columns
month_origin_df = month_origin_df.drop(['fl_month','origin'],axis=1)
month_origin_df = month_origin_df.rename(columns={'dep_delay':'avg_dep_delay_by_month_by_origin_ap','nas_delay':'avg_nas_delay_by_month_by_origin_ap','carrier_delay':'avg_carrier_delay_by_month_by_origin_ap','arr_delay':'avg_arr_delay_by_month_by_origin_ap','cancelled':'cancelled_flights_by_month_by_origin_ap','weather_delay':'avg_weather_delay_by_month_by_origin_ap','security_delay':'security_delay_by_month_by_origin_ap'})
month_origin_df.head()

,avg_dep_delay_by_month_by_origin_ap,avg_arr_delay_by_month_by_origin_ap,avg_carrier_delay_by_month_by_origin_ap,avg_nas_delay_by_month_by_origin_ap,late_aircraft_delay,avg_weather_delay_by_month_by_origin_ap,security_delay_by_month_by_origin_ap,cancelled_flights_by_month_by_origin_ap,combo2


In [27]:
#daily arrival info - unused
daily_arrivals = pd.read_csv('data/new_features/num_arrivals_daily.csv')
daily_departures = pd.read_csv('data/new_features/num_departures_daily.csv')

In [16]:
daily_arrivals.head()

,fl_date,dest,num_flights_arriving
0,2018-01-01,ABE,10
1,2018-01-01,ABI,6
2,2018-01-01,ABQ,77
3,2018-01-01,ABR,2
4,2018-01-01,ABY,2


In [17]:
#tailnumber info - unused
tailnum_df = pd.read_csv('data/features_tailnum_delay_taxi_median_params.csv')
tailnum_df = tailnum_df.rename(columns={'dep_delay':'dep_delay_by_plane','taxi_out':'taxi_out_by_plane','arr_delay':'arr_delay_by_plane','taxi_in':'taxi_in_by_plane'})

In [18]:
tailnum_df.head()

,tail_num,dep_delay_by_plane,taxi_out_by_plane,arr_delay_by_plane,taxi_in_by_plane
0,215NV,-1.0,11.0,-4.0,6.0
1,216NV,-3.0,12.0,-5.0,6.0
2,217NV,-2.0,11.0,-3.0,7.0
3,218NV,-2.0,11.0,-4.0,6.0
4,219NV,-4.0,11.0,-6.0,6.0


In [19]:
#convert time to # of minute since midnight
df['crs_arr_time'] = (df['crs_arr_time']//100)*60 + df['crs_arr_time'] - (df['crs_arr_time']//100)*100
df['crs_dep_time'] = (df['crs_dep_time']//100)*60 + df['crs_dep_time'] - (df['crs_dep_time']//100)*100

In [20]:
#add in fuel and passenger info by carrier
df = df.merge(pass_fuel_df, how = 'left', left_on='mkt_unique_carrier',right_on='mkt_unique_carrier')

In [21]:
#add in departure and passenger info by airport
df = df.merge(departures_df, how = 'left', on = 'origin')

#add in arrival and passenger info by arrival airport
df = df.merge(arrivals_df, how = 'left', on = 'dest')

#add in average departure delay by departure location
df = df.merge(dep_delay, how = 'left', left_on = 'origin', right_index = True)

#add in average arrival delay by arrival location
df = df.merge(arr_delay, how = 'left', left_on = 'origin', right_index = True)

#add in monthly data
df = df.merge(month_dest_df, how = 'left', on = 'combo1')
df = df.merge(month_origin_df, how = 'left', on = 'combo2')

#Below excluded because their addition INCREASED RMSE
# #add in departures & arrivals per day
# df = df.merge(daily_arrivals, how = 'left', on = ['fl_date','dest'])
# df = df.merge(daily_departures, how = 'left', on = ['fl_date','origin'])

# #add in plane level info
# df = df.merge(tailnum_df, how='left', on = 'tail_num')

In [22]:
df = df.drop(['fl_date','tail_num','mkt_carrier_fl_num'], axis = 1)

In [23]:
#create variable where 1 if op_carrier different from mkt_carrier
df['op_unique_carrier_check'] = df['op_unique_carrier'] != df['mkt_unique_carrier']
df['op_unique_carrier_check'] = df['op_unique_carrier_check'].astype('int')

In [24]:
#fill arrival/departure average nans with 0
df = df.fillna(0)

In [25]:
df = df.drop(['origin','dest','op_unique_carrier','combo1','combo2','cancelled_flights','Unnamed: 0'],axis=1)

In [26]:
df.columns

Index(['constant', 'mkt_unique_carrier', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance', 'arr_delay', 'month', 'weekday',
       'avg_distance_per_month_by_carrier', 'avg_dep_delay_by_carrier',
       'total_gallons', 'passengers_by_carrier_per_month',
       'monthly_distance_per_passenger', 'avgfuel_percustomer_perdistance',
       'yearly_departures_per_ap', 'yearly_passengers_per_ap',
       'yearly_arrivals_per_ap', 'yearly_arriving_passengers_per_ap',
       'average_departure_delay_by_ap', 'average_arrival_delay_by_ap_full',
       'avg_arr_delay_by_month_by_dest_ap',
       'avg_carrier_delay_by_month_by_dest_ap',
       'avg_nas_delay_by_month_by_dest_ap', 'late_aircraft_delay_x',
       'avg_weather_delay_by_month_by_dest_ap',
       'security_delay_by_month_by_dest_ap',
       'cancelled_flights_by_month_by_dest_ap',
       'avg_dep_delay_by_month_by_origin_ap',
       'avg_arr_delay_by_month_by_origin_ap',
       'avg_carrier_delay_by_month_by_origin_

In [27]:
df.head()

,constant,mkt_unique_carrier,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,month,weekday,avg_distance_per_month_by_carrier,...,cancelled_flights_by_month_by_dest_ap,avg_dep_delay_by_month_by_origin_ap,avg_arr_delay_by_month_by_origin_ap,avg_carrier_delay_by_month_by_origin_ap,avg_nas_delay_by_month_by_origin_ap,late_aircraft_delay_y,avg_weather_delay_by_month_by_origin_ap,security_delay_by_month_by_origin_ap,cancelled_flights_by_month_by_origin_ap,op_unique_carrier_check
0,1,DL,620,686,66.0,153,-5.0,2,1,1.014802e+08,...,0.006185,11.312448,4.526793,3.995054,3.525556,6.062242,0.645507,0.000000,0.028854,0
1,1,AS,1205,1290,85.0,308,53.0,5,5,3.493391e+07,...,0.019439,8.986239,1.990982,3.388559,2.506986,4.166428,0.081394,0.010356,0.010262,0
2,1,DL,940,1018,78.0,270,-13.0,1,1,1.014802e+08,...,0.005505,9.814683,3.909325,3.649802,3.181944,5.326984,0.321429,0.000000,0.012103,0
3,1,UA,1035,1134,99.0,325,-33.0,11,6,1.013686e+08,...,0.014629,9.555175,5.925672,4.845193,2.577689,5.778856,0.577055,0.000000,0.007230,1
4,1,AA,790,877,87.0,331,-12.0,7,1,1.198062e+08,...,0.037967,14.977925,11.530916,4.291598,3.141092,7.255694,1.582681,0.049149,0.021958,0


In [28]:
dummy_df = pd.get_dummies(df,columns=['mkt_unique_carrier','month','weekday'])

In [29]:
#drop final category of each dummy variable
dummy_df = dummy_df.drop(columns=['month_12','weekday_6','mkt_unique_carrier_WN'])

In [ ]:
dummy_df.to_csv('data/test_df_transformed.csv')